<h1><center>CONNECT FOUR</center></h1>

The objective of this notebook is to describe the process of creating the Connect Four game using the A* and Monte Carlo Tree Search (MCTS) algorithms in the context of the Artificial Intelligence discipline (CC2006).

# STRUCTURE OF THE NOTEBOOK
Initially, the project was conceived not for a Jupyter notebook but rather for a terminal. All the work was first built in Python code within VSCode, and once the results were satisfactory, the migration to the notebook was performed. It is important to emphasize this because the order of the codes presented in the notebook are not the same as we initially thought, although this helps to avoid the use of imports within Python and to minimize duplicated codes and functions that we had previously.

# HEURISTIC
This section of the project describes the creation of the heuristic that will be utilized throughout the project.



## Import
Used libraries for the Heuristic.

In [33]:
import copy
import random
import timeit

## Cor
Class that contains constants to represent ANSI color codes in the terminal. These colors are used in the user's interaction with the terminal and in the different colors used on the board.

In [34]:
class Cor:
    RESET = '\033[0m'
    VERMELHO = '\033[91m'
    AZUL = '\033[94m'
    MAGENTA = '\033[95m'

## Board
The Board contains the games grid and a method to output the grid to the stdout.

In [35]:
class Board:
    def __init__(self):
        self.Grid = [['X' for _ in range(7)] for _ in range(6)]

    def print_grid(self):
        for i in range(6):
            for j in range(7):
                if(self.Grid[i][j] == 'R'):
                    print(Cor.VERMELHO + self.Grid[i][j] + Cor.RESET, end=" ")
                if(self.Grid[i][j] == 'B'):
                    print(Cor.AZUL + self.Grid[i][j] + Cor.RESET, end=" ")
                if(self.Grid[i][j] == 'X'):
                    print(self.Grid[i][j], end=" ")
            print()

## Compute_Score
Calculates the value for a set of pieces, as specified in the project worksheet.
Where:
>
>`if(count_R >= 1 and count_B >= 1):` verifies if there is any blue and red piece.
>
>`if(count_R != 0):` verifies if there is any red pieces.
>
> `elif(count_B != 0):` in case there is no red pieces, this verifies if there is blue pieces.


In [36]:
def Compute_Score(count_R, count_B):
    if(count_R >= 1 and count_B >= 1):
        return 0
    else:
        if(count_R != 0):
            match count_R:
                case 1:
                    return (1)
                case 2:
                    return (10)
                case 3:
                    return (50)
                case 4:
                    return 512
        elif(count_B != 0):
            match count_B:
                case 1:
                    return -1
                case 2:
                    return -10
                case 3:
                    return -50
                case 4:
                    return -512
    return 0

## Is_Red
This function verifies if the piece is red.

In [37]:
def Is_Red(piece):
    if(piece == 'R'):
        return True
    else:
        return False

## Compute_Lines
This function computes all 4 line pieces segments.
> `for i in range(6):` represents the grid lines.
>
> `for j1 in range(0,4,1):` is the segment offset, 4 for each line.
>
> `total_temp = Compute_Score(count_R, count_B)` calculates the score.

In [38]:
def Compute_Lines(Game):
    total = 0
    total_temp = 0
    count_R = 0
    count_B = 0
    for i in range(6):
        for j1 in range(0,4,1):
            for j2 in range(0,4,1):
                if(Game.Grid[i][j1+j2] == 'X'):
                    continue
                elif(Is_Red(Game.Grid[i][j1+j2])):
                    count_R += 1
                else:
                    count_B += 1
            total_temp = Compute_Score(count_R, count_B)
            if(total_temp == 512 or total_temp == -512):
                return total_temp
            total += total_temp
            total_temp = 0
            count_R = 0
            count_B = 0
    return total

## Compute_Columns
This function does the same as `def Compute_Lines(Game):`, but instead of lines it is applied to columns.
>`for j in range(7):` represents the grid columns.
>
>`for i1 in range(0,3,1):` segment offset, 3 for each column.
>
>`total_temp = Compute_Score(count_R, count_B)` calculates the score.

In [39]:
def Compute_Columns(Game):
    total = 0
    total_temp = 0
    count_R = 0
    count_B = 0
    for j in range(7):
        for i1 in range(0,3,1):
            for i2 in range(0,4,1):
                if(Game.Grid[i1+i2][j] == 'X'):
                    continue
                elif(Is_Red(Game.Grid[i1+i2][j])):
                    count_R += 1
                else:
                    count_B += 1
            total_temp = Compute_Score(count_R, count_B)
            if(total_temp == 512 or total_temp == -512):
                return total_temp
            total += total_temp
            total_temp = 0
            count_R = 0
            count_B = 0
    return total

## Compute_Diagonals_Positive
Same as the previous two but directed to diagonals.
>`for i in range(3):` when shown it represents the calculation of a new diagonal, in total there is 4 of them(2 positive and 2 negative).
>
>`total_temp = Compute_Score(count_R, count_B)` represents the score calculation


In [40]:
def Compute_Diagonals_Positive(Game):
    total = 0
    total_temp = 0
    count_R = 0
    count_B = 0
    count_number_fors = 3
    for i in range(3):
        for temp in range (count_number_fors):
            for j in range(4):
                if(Game.Grid[i+j+temp][0+j+temp] == 'X'):
                    continue
                elif(Is_Red(Game.Grid[i+j+temp][0+j+temp])):
                    count_R += 1
                else:
                    count_B += 1
            total_temp = Compute_Score(count_R, count_B)
            if(total_temp == 512 or total_temp == -512):
                return total_temp
            total += total_temp
            total_temp = 0
            count_R = 0
            count_B = 0
        count_number_fors -= 1
    count_number_fors = 3
    for i in range(3):
        for temp in range (count_number_fors):
            for j in range(4):
                if(Game.Grid[j+temp][1+j+temp+i] == 'X'):
                    continue
                elif(Is_Red(Game.Grid[j+temp][1+j+temp+i])):
                    count_R += 1
                else:
                    count_B += 1
            total_temp = Compute_Score(count_R, count_B)
            if(total_temp == 512 or total_temp == -512):
                return total_temp
            total += total_temp
            total_temp = 0
            count_R = 0
            count_B = 0
        count_number_fors -= 1
    count_number_fors = 3
    for i in range(3):
        for temp in range (count_number_fors):
            for j in range(4):
                if(Game.Grid[0+j+temp+i][6-j-temp] == 'X'):
                    continue
                elif(Is_Red(Game.Grid[0+j+temp+i][6-j-temp])):
                    count_R += 1
                else:
                    count_B += 1
            total_temp = Compute_Score(count_R, count_B)
            if(total_temp == 512 or total_temp == -512):
                return total_temp
            total += total_temp
            total_temp = 0
            count_R = 0
            count_B = 0
        count_number_fors -= 1
    count_number_fors = 3
    for i in range(3):
        for temp in range (count_number_fors):
            for j in range(4):
                if(Game.Grid[0+j+temp][5-i-j-temp] == 'X'):
                    continue
                elif(Is_Red(Game.Grid[0+j+temp][5-i-j-temp])):
                    count_R += 1
                else:
                    count_B += 1
            total_temp = Compute_Score(count_R, count_B)
            if(total_temp == 512 or total_temp == -512):
                return total_temp
            total += total_temp
            total_temp = 0
            count_R = 0
            count_B = 0
        count_number_fors -= 1
    count_number_fors = 3
    return total

## Total_Value
Calcullates the total value from the grid by adding the value of lines, columns and diagonals. And if any of theese sums is equal to **512** or **-512**, it means ones of the players has won.
In the end it returns the the addiction between `sum_lines`, `sum_columns` and `sum_diagonals`.

In [41]:
def Total_Value(Game):
    sum_columns = 0
    sum_diagonals = 0
    sum_columns = 0
    sum_lines = Compute_Lines(Game)
    if(sum_lines == 512 or sum_lines == -512):
        return sum_lines
    sum_columns = Compute_Columns(Game)
    if(sum_columns == 512 or sum_columns == -512):
        return sum_columns
    sum_diagonals = Compute_Diagonals_Positive(Game)
    if(sum_diagonals == 512 or sum_diagonals == -512):
        return sum_diagonals
    return (sum_lines + sum_columns + sum_diagonals)

# A STAR
This is the begining of the implementation of A Star algorithm.

## Make_Move
The function computes the move, inputed by the index argument, by the player indentified by the color argument.

In [42]:
def Make_Move(Return_Board, index, color):
    for i in range(5, -1, -1):
        if(Return_Board.Grid[i][index] == 'X'):
            Return_Board.Grid[i][index] = color
            return
    print("You should not be here")

## A_Star
This function implements part of the AI logic using the A* algorithm. It considers possible moves, evaluates their values, and chooses the move with the highest or lowest value to make. All of this with functions previously implented.

In the end, It returns the value of the chosen move, with some adjustments (+16 for **Red** and -16 for **Blue**).

In [44]:
def A_Star(playerColor, game):
    if(playerColor == 'R'):
        best_value = -513 
    else:
        best_value = 513
    best_move = -1
    for i in [i for i in range(7) if temp_game.Grid[0][i] == 'X']:
        a_star_board = Board()
        lista_temp = copy.deepcopy(game.Grid)
        setattr(a_star_board, 'Grid', lista_temp)
        Make_Move(a_star_board, i, playerColor)
        value_temp = Total_Value(a_star_board)
        if(playerColor == 'R' and value_temp > best_value):
            best_value = value_temp
            best_move = i
        elif(playerColor == 'B' and value_temp > best_value):
             best_value = value_temp
            best_move = i
    Make_Move(game, best_move, playerColor)
    if(playerColor == 'R'):
        return best_value + 16
    else:
        return best_value - 16

# MONTE CARLO TREE SEARCH
This section is about the MCTS algorithm implementation.

## Imports
For the MCTS implementation, two libraries were used, `copy` and `random`, already imported in the heuristic.

## Node
Represents a node in the game tree for MCTS.

Here the function `__init__` belongs to the classe node and initializes a node object with 6 atributes:
>`state` represents the state of the game associated with this node.
>
>`visits` tracks how many times this node has been visited during the MCTS algorithm;
>
>`score` represents the cumulative score associated with this node;
>
>`children` a list that stores child nodes from the current node;
>
>`parent` reference to the parent node of current node;
>
>`move` represents the move that led to this node.


In [45]:
class Node:
    def __init__(self, state):
        self.state = state
        self.visits = 0
        self.score = 0
        self.children = []
        self.parent = None
        self.move = -1

## Select
Function that selects the next child node using the Upper Confidence Bound (UCB) policy and the child is chosen with the highest UCB value.
> `max_ucb = -float('inf')` is the initial value for the UCB (smallest value).
>
> `ucb = float('inf')` if the child node hasn´t been found, then the ucb value is equal to infinity in order to force a find of this node.
>
> `ucb = (child.score / child.visits) + (2 * (2 * node.visits / child.visits) ** 0.5)` It is the ucb formula.

In [46]:
def select(node):
    selected_node = None
    max_ucb = -float('inf')
    for child in node.children:
        if child.visits == 0:
            ucb = float('inf')
        else:
            # Fórmula do UCB: exploração (score/visits) + exploração (2 * log(total_visits) / visits) ** 0.5
            ucb = (child.score / child.visits) + (2 * (2 * node.visits / child.visits) ** 0.5)
        if ucb > max_ucb:
            max_ucb = ucb
            selected_node = child
    return selected_node

## Expand
Function with the sole purpose to expand the child from the current node for both players. Also creates child nodes corresponding to each possible move in a column.

In [47]:
def expand(node, jogadas):
    for column in range(7):
        temp_game = copy.deepcopy(node.state)
        if Game_is_Over(temp_game, 'R') or Game_is_Over(temp_game, 'B'):
            break
        if temp_game.Grid[0][column] == 'X':
            for c in jogadas:
                Make_Move(temp_game, column, c)
                new_node = Node(temp_game)
                new_node.parent = node
                new_node.move = column
                node.children.append(new_node)
                temp_game = copy.deepcopy(node.state)

## Simulate
Function that simulates a game starting from a given state for both players until the game ends.
>`column = random.choice(columns)` choses a random available column.
>
>`return Total_Value(temp_game)` returns the total value of the game using a heuristic.

In [48]:
def simulate(state, jogadas):
    temp_game = copy.deepcopy(state)
    while True:
        for c in jogadas:
            columns = [i for i in range(7) if temp_game.Grid[0][i] == 'X']
            if (len(columns) != 0):
                column = random.choice(columns)
                Make_Move(temp_game, column, c)
            else:
                return Total_Value(temp_game)
            if Game_is_Over(temp_game, 'R') or Game_is_Over(temp_game, 'B'):
                return Total_Value(temp_game)

## Backpropagate
Function to update the visit count and score of nodes in the tree after a simulation.

In [49]:
def backpropagate(node, score):
    while node is not None:
        node.visits += 1
        node.score += score
        node = node.parent

## MCTS
The Main function implementing the Monte Carlo Tree Search algorithm, it uses functions like `select`, `expand`, `simulate` and `backpropagate`, that were already implemented and written about, to find the best move possible.
> `node = select(node)` selects the next child node by using UCB.
>
> `expand(node, jogadas)` expands the child from the current node.
>
> `score = simulate(node.state, jogadas)` simulates a game using the current state of the node.
>
> `backpropagate(node, score)` backpropagates the results.

In [50]:
def mcts(state, iterations, player):
    if (player == 1):
        jogadas = ['R', 'B']
    else:
        jogadas = ['B', 'R']
    root = Node(state)
    for _ in range(iterations):
        node = root
        while node.children:
            node = select(node)
        expand(node, jogadas)
        score = simulate(node.state, jogadas)
        backpropagate(node, score)
    best_move = None
    best_score = -float('inf')
    for child in root.children:
        if ((child.visits > 0) and (child.score / child.visits) > best_score):
            best_move = child.move
            best_score = (child.score / child.visits)
    return best_move

## Make_Move
Computes the move, inputed by the index argument, by the player indentified by the color argument

In [51]:
def Make_Move(Return_Board, index, color):
    for i in range(5, -1, -1):
        if(Return_Board.Grid[i][index] == 'X'):
            Return_Board.Grid[i][index] = color
            return
    print("You should not be here")

## Check_Line
This function checks for a winning line in the game grid by checking if all positions in one row are occupied by the same player's piece.

In [52]:
def Check_Line(Game, i_plus, j_plus, player):
    for i in range(4):
        if(Game.Grid[i + i_plus][0 + j_plus] == player and Game.Grid[i + i_plus][1 + j_plus] == player and Game.Grid[i + i_plus][2 + j_plus] == player and Game.Grid[i + i_plus][3 + j_plus] == player):
            return True
    return False

## Check_Column
Simple checking Column algorithm that works the same way as `Check_line`, but instead of lines it is columns.

In [53]:
def Check_Column(Game, i_plus, j_plus, player):
    for j in range(4):
        if(Game.Grid[0 + i_plus][j + j_plus] == player and Game.Grid[1 + i_plus][j + j_plus] == player and Game.Grid[2 + i_plus][j + j_plus] == player and Game.Grid[3 + i_plus][j + j_plus] == player):
            return True
    return False

## Check_Diagonal
Same as the previous two, `Check_Line` and `Check_Column`, but here we are dealing with the diagonals.

In [54]:
def Check_Diagonal(Game, i_plus, j_plus, player): # Simple checking Diagonal algorithm
    if((Game.Grid[0 + i_plus][0 + j_plus] == player and Game.Grid[1 + i_plus][1 + j_plus] == player and Game.Grid[2 + i_plus][2 + j_plus] == player and Game.Grid[3 + i_plus][3 + j_plus] == player) or (Game.Grid[0 + i_plus][3 + j_plus] == player and Game.Grid[1 + i_plus][2 + j_plus] == player and Game.Grid[2 + i_plus][1 + j_plus] == player and Game.Grid[3 + i_plus][0 + j_plus] == player)):
        return True
    return False

## Game_is_Over
Function that checks if the game is over, by using the functions that check for a win.
> `if(Check_Column(test, i, j, color)):` here it checks the Columns.
>
> `if(Check_Line(test, i, j, color)):` here it checks the Lines.
>
> `if(Check_Diagonal(test, i, j, color)):` here it checks the Diagonals.

In [55]:
def Game_is_Over(test, color): #Checks if the Game is over
    for i in range(3):
        for j in range(4):
            if(Check_Column(test, i, j, color)): # Here it checks the Columns
                #print(color +" Wins")
                #test.print_grid()
                return True
            if(Check_Line(test, i, j, color)): # Here it checks the Lines
                #print(color + " Wins")
                #test.print_grid()
                return True
            if(Check_Diagonal(test, i, j, color)): # Here it checks the Diagonals
                #print(color + " Wins")
                #test.print_grid()
                return True
    return False

# BOARD
Contains all the functions that are necessary to have a connect four game.

In [56]:
VALOR = 10000

The variable `Valor` represent the number of iterations or simulations that the MCTS algorithm will perform.

## Take_Int_Input
This function takes input from the stdin and makes sure the input is an int.

In [57]:
def Take_Int_input():
    while(True):
        try:
            ret = int(input('Choose your move: '))
            return ret
        except ValueError:
            continue

## Red_moves
This function computes the move for the red player.
>`list_possible_moves = [False for _ in range(7)]` list of possible moves, for now its entirety is false;
>
>`for i in range(7):` checks if all columns are filled (in other words if a move is possible), if it is, print the corresponding number above the column;
>
>`list_possible_moves[i] = True` also sets the possible moves list in its i possition to true and `print(" ", end = " ")` or else sets it to false just to make sure;
>
>`if(red_human_player):` checks if Red player is human or not, the negative case is not currently implemented;
>
>`if(not(True in list_possible_moves)):` checks if the result is a tie;
>
> `move = Take_Int_input() - 1` user input, if the move is possible it calls the Make_Move function, if not outputs error and returns to here.

In [72]:
def Red_moves(Return_Board, red_human_player, heuristic, cpu):
    print("Reds Turn")
    print()
    list_possible_moves = [False for _ in range(7)]
    for i in range(7):
            if(Return_Board.Grid[0][i] == 'X'):
                if (red_human_player):
                    print(i+1, end= " ")
                list_possible_moves[i] = True
            else:
                if (red_human_player):
                    print(" ", end = " ")
                list_possible_moves[i] = False
    if(red_human_player):
        print()
        Return_Board.print_grid()
        if(not(True in list_possible_moves)):
            print("Game is a Tie")
            exit()
        while(True):
            move = Take_Int_input() - 1
            if(move <= 6 and move >= 0):
                if(list_possible_moves[move]):
                    Make_Move(Return_Board, move, 'R')
                    break
        heuristic = Total_Value(Return_Board)
    else:
        print()
        #Return_Board.print_grid()
        if (cpu == 1):
            heuristic = A_Star('R', Return_Board)
        elif (cpu == 2):
            move = mcts(Return_Board, VALOR, 1)
            Make_Move(Return_Board, move, 'R')
        elif (cpu == 3):
            move = call_alpha_beta(Return_Board, 'R', 5)
            Make_Move(Return_Board, move, 'R')
        Return_Board.print_grid()
        print()
    #system('clear')
    return heuristic

## Blue_moves
This function has similar functionality to `Red_moves()` but for the blue player, it also takes the same arguments and apply them to the blue player instead.

In [73]:
def Blue_moves(Return_Board:Board, blue_human_player, heuristic, cpu):
    print("Blues Turn")
    print()
    list_possible_moves = [False for _ in range(7)]
    for i in range(7):
            if(Return_Board.Grid[0][i] == 'X'):
                if (blue_human_player):
                    print(i+1, end= " ")
                list_possible_moves[i] = True
            else:
                if (blue_human_player):
                    print(" ", end = " ")
                list_possible_moves[i] = False
    print()
    if(blue_human_player):
        Return_Board.print_grid()
        if(not(True in list_possible_moves)):
            print("Game is a Tie")
            exit()
        while(True):
            move = Take_Int_input() - 1
            if(move <= 6 and move >= 0):
                if(list_possible_moves[move]):
                    Make_Move(Return_Board, move, 'B')
                    break
    else:
        if (cpu == 1):
            heuristic = A_Star('B', Return_Board)
        elif(cpu == 2):
            move = mcts(Return_Board, VALOR, 2)
            Make_Move(Return_Board, move, 'B')
        elif(cpu == 3):
            move = call_alpha_beta(Return_Board, 'B', 5)
            Make_Move(Return_Board, move, 'B')
        Return_Board.print_grid()
        print()
    #system('clear')
    return heuristic

## Game_is_Over
This function checks if the game is over. It takes two arguments: `test`, which is the game board object, and `color`, which is the color of the player whose turn it is.
>`if(Check_Column(test, i, j, color)):` it checks the columns;
>
>`if(Check_Line(test, i, j, color)):` it checks the lines;
>
>`if(Check_Diagonal(test, i, j, color)):` it checks the Diagonals.

In [62]:
def Game_is_Over(test, color):
    for i in range(3):
        for j in range(4):
            if(Check_Column(test, i, j, color)):
                #print(color +" Wins")
                #test.print_grid()
                return True
            if(Check_Line(test, i, j, color)):
                #print(color + " Wins")
                #test.print_grid()
                return True
            if(Check_Diagonal(test, i, j, color)): # Here it checks the Diagonals
                #print(color + " Wins")
                #test.print_grid()
                return True
    return False

## Check_Line
This function checks for a winning condition in a horizontal line.

In [63]:
def Check_Line(Game, i_plus, j_plus, player):
    for i in range(4):
        if(Game.Grid[i + i_plus][0 + j_plus] == player and Game.Grid[i + i_plus][1 + j_plus] == player and Game.Grid[i + i_plus][2 + j_plus] == player and Game.Grid[i + i_plus][3 + j_plus] == player):
            return True
    return False

## Check_Column
This function checks for a winning condition in a vertical line.

In [64]:
def Check_Column(Game, i_plus, j_plus, player):
    for j in range(4):
        if(Game.Grid[0 + i_plus][j + j_plus] == player and Game.Grid[1 + i_plus][j + j_plus] == player and Game.Grid[2 + i_plus][j + j_plus] == player and Game.Grid[3 + i_plus][j + j_plus] == player):
            return True
    return False

## Check_Diagonal
This function checks for a winning condition in a diagonal line.

In [65]:
def Check_Diagonal(Game, i_plus, j_plus, player): # Simple checking Diagonal algorithm
    if((Game.Grid[0 + i_plus][0 + j_plus] == player and Game.Grid[1 + i_plus][1 + j_plus] == player and Game.Grid[2 + i_plus][2 + j_plus] == player and Game.Grid[3 + i_plus][3 + j_plus] == player) or (Game.Grid[0 + i_plus][3 + j_plus] == player and Game.Grid[1 + i_plus][2 + j_plus] == player and Game.Grid[2 + i_plus][1 + j_plus] == player and Game.Grid[3 + i_plus][0 + j_plus] == player)):
        return True
    return False

# ALPHA BETA


This section is about the alpha beta algorithm implementation.

# Reverse_color

In [ ]:
Return the other players color.

In [66]:
AB_VAL = 100000

def reverse_color(color):
    if(color == 'R'):
        return('B')
    else:
        return('R')


## Alpha Beta


Alpha–beta pruning is a search algorithm aiming to reduce the number of nodes evaluated by the minimax algorithm in its search tree. It's commonly employed in machine playing of two-player combinatorial games like Tic-tac-toe, Chess, Connect 4, etc.

In [67]:
def alpha_beta(game, maxPlayer, cdepth, color, alpha, beta):
    if cdepth == 0 or Game_is_Over(game, 'R') or Game_is_Over(game, 'L'):
        return Total_Value(game)
    elif(maxPlayer):
        maxV = -AB_VAL
        possiblemoves = [i for i in range(7) if game.Grid[0][i] == 'X']
        for i in possiblemoves:
            temp_game = copy.deepcopy(game)
            Make_Move(temp_game, i, color)
            #temp_game.print_grid()
            eva = alpha_beta(temp_game, False, cdepth-1, reverse_color(color), alpha, beta)
            maxV = max(maxV, eva)
            alpha = max(maxV, alpha)
            if (beta <= alpha):
                break
        return maxV
    else:
        minV = AB_VAL
        possiblemoves = [i for i in range(7) if game.Grid[0][i] == 'X']
        for i in possiblemoves:
            temp_game = copy.deepcopy(game)
            Make_Move(temp_game, i, color)
            #temp_game.print_grid()
            eva = alpha_beta(temp_game, True, cdepth-1, reverse_color(color), alpha, beta)
            minV = min(minV, eva)
            beta = min(minV, beta)
            if (beta <= alpha):
                break
        return minV



## Call Alpha Beta

Calls the alpha beta function for each possible move of a given state and returns the one with the best outcome.

In [70]:
def call_alpha_beta(game, color, depth):
    if(color == 'R'):
        maxscore = -AB_VAL
    else:
        maxscore = AB_VAL
    best_move = -1
    possiblemoves = [i for i in range(7) if game.Grid[0][i] == 'X']
    for x in possiblemoves:
        temp_game = copy.deepcopy(game)
        Make_Move(temp_game, x, color)
        score = alpha_beta(temp_game, True, depth, reverse_color(color), -AB_VAL, AB_VAL)
        if(color == 'R'):
            if(maxscore < score):
                maxscore = score
                best_move = x
        else:
            if(maxscore > score):
                maxscore = score
                best_move = x
    return best_move

# MAIN
Contains the main function that allows to run and play the connect four game.

## Imports
For this file it is only needed functions that were already defined in `Board`, `heuristic` and `a_star`.

## Main
Used to run the program.
>`while(True):` used for playing the game, first comes reds turn then if the game is not over comes blues turn and again if it's not over it loops back to reds turn.


In [75]:
def main():
    print("""

 ░▒▓██████▓▒░  ░▒▓██████▓▒░ ░▒▓███████▓▒░ ░▒▓███████▓▒░ ░▒▓████████▓▒░░▒▓██████▓▒░░▒▓████████▓▒░      ░▒▓█▓▒░░▒▓█▓▒░
░▒▓█▓▒░░▒▓█▓▒░░▒▓█▓▒░░▒▓█▓▒░░▒▓█▓▒░░▒▓█▓▒░░▒▓█▓▒░░▒▓█▓▒░░▒▓█▓▒░      ░▒▓█▓▒░░▒▓█▓▒░  ░▒▓█▓▒░          ░▒▓█▓▒░░▒▓█▓▒░
░▒▓█▓▒░       ░▒▓█▓▒░░▒▓█▓▒░░▒▓█▓▒░░▒▓█▓▒░░▒▓█▓▒░░▒▓█▓▒░░▒▓█▓▒░      ░▒▓█▓▒░         ░▒▓█▓▒░          ░▒▓█▓▒░░▒▓█▓▒░
░▒▓█▓▒░       ░▒▓█▓▒░░▒▓█▓▒░░▒▓█▓▒░░▒▓█▓▒░░▒▓█▓▒░░▒▓█▓▒░░▒▓██████▓▒░ ░▒▓█▓▒░         ░▒▓█▓▒░          ░▒▓████████▓▒░
░▒▓█▓▒░       ░▒▓█▓▒░░▒▓█▓▒░░▒▓█▓▒░░▒▓█▓▒░░▒▓█▓▒░░▒▓█▓▒░░▒▓█▓▒░      ░▒▓█▓▒░         ░▒▓█▓▒░                 ░▒▓█▓▒░
░▒▓█▓▒░░▒▓█▓▒░░▒▓█▓▒░░▒▓█▓▒░░▒▓█▓▒░░▒▓█▓▒░░▒▓█▓▒░░▒▓█▓▒░░▒▓█▓▒░      ░▒▓█▓▒░░▒▓█▓▒░  ░▒▓█▓▒░                 ░▒▓█▓▒░
 ░▒▓██████▓▒░  ░▒▓██████▓▒░ ░▒▓█▓▒░░▒▓█▓▒░░▒▓█▓▒░░▒▓█▓▒░░▒▓████████▓▒░░▒▓██████▓▒░   ░▒▓█▓▒░                 ░▒▓█▓▒░


    """)
    print("Options")
    print("[1]Play as Red")
    print("[2]Play as Blue")
    print("[3]Two player mode")
    print("[4]CPU vs CPU")
    while(True):
        try:
            opt = int(input())
            break
        except ValueError:
            continue
    match opt:
        case 1:
            red_human_player = True
            blue_human_player = False
        case 2:
            red_human_player = False
            blue_human_player = True
        case 3:
            red_human_player = True
            blue_human_player = True
        case 4:
            red_human_player = False
            blue_human_player = False
    print("Choose an algorithm: ")
    print("[1] A * Star")
    print("[2] MCTS")
    print("[3] Alpha Beta")
    while(True):
        try:
            opc = int(input())
            break
        except ValueError:
            continue
    heuristic = 0
    print(Cor.MAGENTA + "Start!" + Cor.RESET)
    test = Board()
    while(True):
        start = timeit.default_timer()
        heuristic = Red_moves(test, red_human_player, heuristic, opc)
        tempo = timeit.default_timer() - start
        if (not red_human_player):
            print("Tempo de execução: " + str(tempo))
            print()
        if(Game_is_Over(test, 'R')):
            break
        start = timeit.default_timer()
        heuristic = Blue_moves(test, blue_human_player,heuristic, opc)
        tempo = timeit.default_timer() - start
        if (not blue_human_player):
            print("Tempo de execução: " + str(tempo))
            print()
        if(Game_is_Over(test, 'B')):
            break
    print()
    if(Game_is_Over(test, 'R')):
        print ("RED WINS!")
    if(Game_is_Over(test, 'B')):
        print ("BLUE WINS!")
    if(not(Game_is_Over(test, 'R') or Game_is_Over(test, 'B'))):
        print("DRAW!")
    test.print_grid()
main()

 

 ░▒▓██████▓▒░  ░▒▓██████▓▒░ ░▒▓███████▓▒░ ░▒▓███████▓▒░ ░▒▓████████▓▒░░▒▓██████▓▒░░▒▓████████▓▒░      ░▒▓█▓▒░░▒▓█▓▒░ 
░▒▓█▓▒░░▒▓█▓▒░░▒▓█▓▒░░▒▓█▓▒░░▒▓█▓▒░░▒▓█▓▒░░▒▓█▓▒░░▒▓█▓▒░░▒▓█▓▒░      ░▒▓█▓▒░░▒▓█▓▒░  ░▒▓█▓▒░          ░▒▓█▓▒░░▒▓█▓▒░ 
░▒▓█▓▒░       ░▒▓█▓▒░░▒▓█▓▒░░▒▓█▓▒░░▒▓█▓▒░░▒▓█▓▒░░▒▓█▓▒░░▒▓█▓▒░      ░▒▓█▓▒░         ░▒▓█▓▒░          ░▒▓█▓▒░░▒▓█▓▒░ 
░▒▓█▓▒░       ░▒▓█▓▒░░▒▓█▓▒░░▒▓█▓▒░░▒▓█▓▒░░▒▓█▓▒░░▒▓█▓▒░░▒▓██████▓▒░ ░▒▓█▓▒░         ░▒▓█▓▒░          ░▒▓████████▓▒░ 
░▒▓█▓▒░       ░▒▓█▓▒░░▒▓█▓▒░░▒▓█▓▒░░▒▓█▓▒░░▒▓█▓▒░░▒▓█▓▒░░▒▓█▓▒░      ░▒▓█▓▒░         ░▒▓█▓▒░                 ░▒▓█▓▒░ 
░▒▓█▓▒░░▒▓█▓▒░░▒▓█▓▒░░▒▓█▓▒░░▒▓█▓▒░░▒▓█▓▒░░▒▓█▓▒░░▒▓█▓▒░░▒▓█▓▒░      ░▒▓█▓▒░░▒▓█▓▒░  ░▒▓█▓▒░                 ░▒▓█▓▒░ 
 ░▒▓██████▓▒░  ░▒▓██████▓▒░ ░▒▓█▓▒░░▒▓█▓▒░░▒▓█▓▒░░▒▓█▓▒░░▒▓████████▓▒░░▒▓██████▓▒░   ░▒▓█▓▒░                 ░▒▓█▓▒░ 
                                                                                                                     
                                                     